# 0.Import libraries

In [1]:
import pandas as pd

# 2. Data preprocessing

In [2]:
df = pd.read_csv("../data/imdb.csv")
df.head()

,Name,Date,Rate,Votes,Genre,Duration,Type,Certificate,Episodes,Nudity,Violence,Profanity,Alcohol,Frightening
0,No Time to Die,2021,7.6,"107,163","Action, Adventure, Thriller",163,Film,PG-13,-,Mild,Moderate,Mild,Mild,Moderate
1,The Guilty,2021,6.3,"64,375","Crime, Drama, Thriller",90,Film,R,-,None,None,Severe,None,Moderate
2,The Many Saints of Newark,2021,6.4,"27,145","Crime, Drama",120,Film,R,-,Moderate,Severe,Severe,Moderate,Moderate
3,Venom: Let There Be Carnage,2021,6.4,"30,443","Action, Adventure, Sci-Fi",97,Film,PG-13,-,None,Moderate,Moderate,Mild,Moderate
4,Dune,2021,8.3,"84,636","Action, Adventure, Drama",155,Film,PG-13,-,None,Moderate,None,Mild,Moderate


In [3]:
df.shape

(6178, 14)

Dữ liệu có 6178 dòng và 14 cột

**Giải thích ý nghĩa từng cột:**
- `Name`: Tên phim
- `Date`: Ngày phát hành
- `Rate`: Điểm imdb trung bình của phim
- `Votes`: Số lượng người đánh giá phim
- `Genre`: Thể loại phim
- `Duration`: Độ dài phim
- `Type`: Loại phim, gồm `Film` - Phim lẻ và `Series` - Phim bộ
- `Certificate`: Nhãn phim
- `Episodes`: Số lượng tập (chỉ đối với các dòng có `Type` là `Series`)
- `Nudity`: Mức độ các cảnh 'nóng' trong phim
- `Violence`: Mức độ các cảnh bạo lực trong phim
- `Profanity`: Mức độ cảc cảnh chửi rủa trong phim
- `Alcohol`: Mức độ các cảnh xuất hiện cồn trong phim
- `Frightening`: mức độ các cảnh gây sợ hãi trong phim

Các cột `Nudity, Violence, Profanity, Alcohol, Frightening` có chung bộ giá trị đánh giá mức độ gồm:
- `None`: Hầu như không có
- `Mild`: Nhẹ
- `Moderate`: Vừa phải
- `Severe`: Nặng
- `No Rate`: Không có đánh giá của người xem ở khía cạnh này

Trước khi tiến hành tiền xử lí dữ liệu, như đã đề cập trước ở phần `1. Data Collecting`, nhóm chỉ tiến hành xử lí và phân tích dữ liệu của các phim lẻ (`Film`), không phân tích dữ liệu các phim bộ (`Series`), do đó nhóm sẽ loại bỏ các dòng có giá trị `Series` của cột `Type`

In [4]:
df = df[df["Type"] == "Film"]

In [5]:
df["Episodes"].value_counts()

-    4446
Name: Episodes, dtype: int64

Sau khi chỉ giữ lại các `Film` thì ở cột `Episodes` chỉ số lượng tập của phim lúc này chỉ chứa giá trị '`-`', do đó nhóm sẽ loại bỏ luôn cột này

In [6]:
df.drop("Episodes", axis=1, inplace=True)

In [7]:
df.shape

(4446, 13)

Sau khi xóa các dòng `Series` và cột `Episodes`, dữ liệu còn lại 4446 dòng, 13 cột

## 2.1. Missing value


In [8]:
df.isna().sum()

Name           0
Date           0
Rate           0
Votes          0
Genre          0
Duration       0
Type           0
Certificate    0
Nudity         0
Violence       0
Profanity      0
Alcohol        0
Frightening    0
dtype: int64

Kiểm tra sơ bằng hàm `isna()`, không có ô dữ liệu nào bị thiếu (không có giá trị). Tuy nhiên khi quan sát dữ liệu, nhóm nhận thấy ở một số cột có các giá trị tượng trưng cho không có dữ liệu như `"None"`, `"No Rate"`

### 2.1.1. Cột `Rate` và cột `Votes`



In [9]:
df["Rate"].value_counts().sort_index().index

Index(['1.2', '1.9', '2.0', '2.3', '2.4', '2.5', '2.7', '2.8', '2.9', '3.0',
       '3.1', '3.2', '3.3', '3.4', '3.5', '3.7', '3.8', '3.9', '4.0', '4.1',
       '4.2', '4.3', '4.4', '4.5', '4.6', '4.7', '4.8', '4.9', '5.0', '5.1',
       '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.8', '5.9', '6.0', '6.1',
       '6.2', '6.3', '6.4', '6.5', '6.6', '6.7', '6.8', '6.9', '7.0', '7.1',
       '7.2', '7.3', '7.4', '7.5', '7.6', '7.7', '7.8', '7.9', '8.0', '8.1',
       '8.2', '8.3', '8.4', '8.5', '8.6', '8.7', '8.8', '8.9', '9.0', '9.2',
       '9.3', 'No Rate'],
      dtype='object')

In [10]:
df["Votes"].value_counts().sort_index().index

Index(['1,000', '1,015', '1,023', '1,031,650', '1,036,534', '1,044',
       '1,045,683', '1,047,410', '1,049,398', '1,054',
       ...
       '99,495', '99,578', '99,655', '99,665', '99,670', '99,760', '990,836',
       '993,069', '995', 'No Votes'],
      dtype='object', length=3439)

In [11]:
df[df["Rate"] == "No Rate"].equals(df[df["Votes"] == "No Votes"])

True

In [12]:
df[df["Rate"] == "No Rate"]["Rate"].count()

145

Kiểm tra hai cột `Rate` và `Votes`, nhóm nhận thấy cột `Rate` có giá trị `No Rate` tượng trưng cho giá trị rỗng, cột `Votes` có giá trị `No Votes` tượng trưng cho giá trị rỗng. Đồng thời các dòng có giá trị `No Rate` cũng chính là các dòng có giá trị `No Votes`. Do chỉ có 145 dòng này, nhóm quyết định xóa các dòng này khỏi dataframe

In [13]:
df = df[df["Rate"] != "No Rate"]

### 2.1.2. Cột Duration

In [14]:
df["Duration"].value_counts().sort_index().index

Index(['100 ', '101 ', '102 ', '103 ', '104 ', '105 ', '106 ', '107 ', '108 ',
       '109 ',
       ...
       '91 ', '92 ', '93 ', '94 ', '95 ', '96 ', '97 ', '98 ', '99 ', 'None'],
      dtype='object', length=150)

In [15]:
df[df["Duration"] == "None"]

,Name,Date,Rate,Votes,Genre,Duration,Type,Certificate,Nudity,Violence,Profanity,Alcohol,Frightening
241,Hotel Transylvania: Transformania,2022,6.9,316,"Animation, Adventure, Comedy",None,Film,PG,None,Severe,Severe,Severe,Severe
1915,Unaired Game of Thrones Prequel Pilot,2019,6.6,90,Fantasy,None,Film,None,No Rate,No Rate,No Rate,No Rate,No Rate
5008,My Struggle,2021,8.7,74,Biography,None,Film,None,No Rate,No Rate,No Rate,No Rate,No Rate
6158,My Struggle,2021,8.7,74,Biography,None,Film,None,No Rate,No Rate,No Rate,No Rate,No Rate


Cột `Duration` chỉ có 4 giá trị `None`. Nhóm quyết định xóa cột này

### 2.1.3. Cột Certificate

In [16]:
df["Certificate"].value_counts()

R            1873
PG-13        1132
PG            524
Not Rated     189
None          182
TV-MA         109
G              70
Approved       42
Unrated        40
TV-14          35
Passed         24
NC-17          14
TV-PG          13
E              12
(Banned)       10
TV-G           10
GP              6
X               6
TV-Y7           5
M               3
M/PG            2
Name: Certificate, dtype: int64

In [17]:
df[df["Certificate"] == "Not Rated"]

,Name,Date,Rate,Votes,Genre,Duration,Type,Certificate,Nudity,Violence,Profanity,Alcohol,Frightening
237,Busanhaeng,2016,7.6,"190,528","Action, Horror, Thriller",118,Film,Not Rated,None,Severe,Mild,None,Severe
266,Batoru rowaiaru,2000,7.6,"176,302","Action, Adventure, Drama",114,Film,Not Rated,Mild,Severe,Mild,Mild,Severe
310,Black as Night,2021,3.9,"1,087","Drama, Horror, Thriller",87,Film,Not Rated,None,Mild,Moderate,Moderate,Moderate
340,Rakka,2017,7.4,"7,037","Short, Horror, Sci-Fi",22,Film,Not Rated,No Rate,Severe,Moderate,Mild,No Rate
351,The Manor,2021,5.3,"1,677",Horror,81,Film,Not Rated,None,None,Mild,None,Mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6109,Anatomy of a Murder,1959,8.0,"63,564","Drama, Mystery",161,Film,Not Rated,None,Mild,Mild,Mild,Mild
6149,Brawl in Cell Block 99,2017,7.2,"62,894","Action, Crime, Drama",132,Film,Not Rated,Mild,Severe,Moderate,Mild,Severe
6154,Dalkomhan insaeng,2005,7.5,"38,677","Action, Crime, Drama",119,Film,Not Rated,None,Severe,Severe,Mild,Moderate
6171,Chugyeokja,2008,7.8,"62,423","Action, Crime, Thriller",125,Film,Not Rated,Mild,Severe,Moderate,Mild,Severe


Quan sát các dòng chứa giá trị `"Not Rated"` ở cột `"Certificate"`, nhóm nhận thấy ở các cột khác tương đối đủ giá trị, do đó nhóm quyết định giữ lại các dòng này, ý nghĩa của giá trị `"Not Rated"` sẽ được quyết định sau 

In [18]:
df[df["Certificate"] == "None"][["Nudity", "Violence", "Profanity", "Alcohol", "Frightening"]].value_counts()

Nudity   Violence  Profanity  Alcohol  Frightening
No Rate  No Rate   No Rate    No Rate  No Rate        182
dtype: int64

Quan sát các dòng chứa giá trị `"None"` ở cột `"Certificate"`, nhóm nhận thấy các cột đánh giá như `"Nudity"`, `"Violence"` đều mang giá trị `"No Rate"`. Do đó các dòng này có thể xem như dữ liệu khuyết của cột `"Certificate"`, nhóm quyết định xóa bỏ các dòng này

In [19]:
df = df[df["Certificate"] != "None"]

### 2.1.4. Các cột đánh giá `"Nudity"`, `"Violence"`, `"Profanity"`, `"Alcohol"`, `"Frightening"`

In [37]:
df[(df["Nudity"] == "No Rate") 
   | (df["Violence"] == "No Rate") 
   | (df["Profanity"] == "No Rate") 
   | (df["Alcohol"] == "No Rate") 
   | (df["Frightening"] == "No Rate")
][
    ["Nudity", "Violence", "Profanity", "Alcohol", "Frightening"]
]

,Nudity,Violence,Profanity,Alcohol,Frightening
57,None,Severe,Severe,No Rate,Severe
194,No Rate,Moderate,Moderate,Mild,Moderate
223,No Rate,No Rate,No Rate,No Rate,No Rate
235,No Rate,No Rate,No Rate,No Rate,No Rate
340,No Rate,Severe,Moderate,Mild,No Rate
...,...,...,...,...,...
5785,No Rate,No Rate,No Rate,No Rate,No Rate
5798,Mild,Moderate,Mild,No Rate,Moderate
5959,No Rate,No Rate,No Rate,No Rate,No Rate
6078,No Rate,No Rate,No Rate,No Rate,No Rate


Các giá trị `"No Rate"` của các cột đánh giá thường xuất hiện cùng nhau. Sau khi thống nhất, nhóm quyết định chỉ giữ lại các dòng có từ 2 cột đồng thời mang giá trị `"No Rate"` trở xuống, bỏ các dòng đồng thời mang từ 3 giá trị `"No Rate"` trở lên

In [49]:
data = df[(df["Nudity"] == "No Rate") 
   | (df["Violence"] == "No Rate") 
   | (df["Profanity"] == "No Rate") 
   | (df["Alcohol"] == "No Rate") 
   | (df["Frightening"] == "No Rate")
][
    ["Nudity", "Violence", "Profanity", "Alcohol", "Frightening"]
].apply(lambda x : x.value_counts()["No Rate"], axis = 1) 

data[data >= 3].index

df = df[~df.index.isin(data[data >= 3].index)]